# Setup

In [1]:
import pandas as pd

TRANSACTION_INDEX = ['TPN_ID']

In [2]:
original = pd.read_csv("./data/original_sorted.csv")

In [3]:
original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5568 entries, 0 to 5567
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   RC#          5568 non-null   int64 
 1   RC Name      5568 non-null   object
 2   Category     5568 non-null   int64 
 3   Vendor#      5568 non-null   int64 
 4   Vendor Name  5568 non-null   object
 5   Address 1    5544 non-null   object
 6   Address 2    1290 non-null   object
 7   City         5469 non-null   object
 8   State        5371 non-null   object
 9   ZIP code     5557 non-null   object
 10  Amount       5568 non-null   int64 
 11  TPN_ID       5568 non-null   object
dtypes: int64(4), object(8)
memory usage: 522.1+ KB


In [4]:
updated = pd.read_csv("./data/updated_sorted.csv").set_index(TRANSACTION_INDEX)

In [5]:
updated.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5920 entries, 1-6000-100877-1 to 94-6900-328199-1
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   RC#          5920 non-null   int64 
 1   RC Name      5920 non-null   object
 2   Category     5920 non-null   int64 
 3   Vendor#      5920 non-null   int64 
 4   Vendor Name  5920 non-null   object
 5   Address 1    5896 non-null   object
 6   Address 2    1359 non-null   object
 7   City         5821 non-null   object
 8   State        5716 non-null   object
 9   ZIP code     5909 non-null   object
 10  Amount       5920 non-null   int64 
dtypes: int64(4), object(7)
memory usage: 555.0+ KB


# Top-line numbers

In [6]:
original_sum = original.Amount.sum()
updated_sum = updated.Amount.sum()

updated_sum - original_sum

30599050

In [7]:
new_transactions = len(updated) - len(original)
new_transactions

352

In [8]:
diff = pd.concat([original, updated]).drop_duplicates(keep=False)

In [9]:
total_transactions_changed = len(diff)
total_transactions_changed

11488

In [10]:
original_transactions_changed = total_transactions_changed - new_transactions
original_transactions_changed

11136

# Merge transactions

In [11]:
merged_transactions = original.merge(updated, on=TRANSACTION_INDEX, suffixes=('_o', '_u'))
merged_transactions['AmountDiff'] = merged_transactions.Amount_u - merged_transactions.Amount_o

# Largest amounts - change from original to updated

In [12]:
merged_transactions.sort_values(by=['AmountDiff'], ascending=False).head(5)

,RC#_o,RC Name_o,Category_o,Vendor#_o,Vendor Name_o,Address 1_o,Address 2_o,City_o,State_o,ZIP code_o,...,Category_u,Vendor#_u,Vendor Name_u,Address 1_u,Address 2_u,City_u,State_u,ZIP code_u,Amount_u,AmountDiff
4768,85,SOMD Administration,7300,424204,BPA II LTD,1468 WEST 9TH STREET SUITE 835,NaN,CLEVELAND,OH,44113,...,7300,424204,BPA II LTD,1468 WEST 9TH STREET SUITE 835,NaN,CLEVELAND,OH,44113,10951439,2584599
5304,92,Business and Auxiliary Services,7300,505445,WYNDHAM PITTSBURGH UNIVERSITY CENTER,545 E JOHN CARPENTER FREEWAY STE 1300,NaN,IRVING,TX,75062,...,7300,505445,WYNDHAM PITTSBURGH UNIVERSITY CENTER,545 E JOHN CARPENTER FREEWAY STE 1300,NaN,IRVING,TX,75062,11657790,2023060
4414,83,General University,7700,204776,"MARSH USA, INC",P O BOX 281915,NaN,ATLANTA,GA,30384,...,7700,204776,"MARSH USA, INC",P O BOX 281915,NaN,ATLANTA,GA,30384,6071082,1735802
5310,92,Business and Auxiliary Services,7300,687995,MWK FORBES II LLC,3341 FORBES AVENUE,NaN,PITTSBURGH,PA,15213,...,7300,687995,MWK FORBES II LLC,3341 FORBES AVENUE,NaN,PITTSBURGH,PA,15213,6280388,1112046
3799,67,Facilities Management,7000,103658,DUQUESNE LIGHT COMPANY,PO BOX 830012,NaN,BALTIMORE,MD,21283-0012,...,7000,103658,DUQUESNE LIGHT COMPANY,PO BOX 830012,NaN,BALTIMORE,MD,21283-0012,24578216,747194


# Largest amounts - only appearing in updated

In [13]:
new_transactions = original.merge(updated.drop_duplicates(), on=TRANSACTION_INDEX, how='right', indicator=True)
new_transactions.fillna(value={"Amount_x": 0}, inplace=True)
new_transactions['AmountDiff'] = new_transactions.Amount_y - new_transactions.Amount_x

In [14]:
display_columns = ['RC#_y', 'RC Name_y', 'Category_y', 'Vendor#_y', 'Vendor Name_y', 'Address 1_y', 'Address 2_y', 'City_y', 'State_y', 'ZIP code_y', 'Amount_y', 'TPN_ID', '_merge', 'AmountDiff']
new_transactions[new_transactions._merge == "right_only"].sort_values(by=['AmountDiff'], ascending=False).head(5)[display_columns]

,RC#_y,RC Name_y,Category_y,Vendor#_y,Vendor Name_y,Address 1_y,Address 2_y,City_y,State_y,ZIP code_y,Amount_y,TPN_ID,_merge,AmountDiff
469,6,Kenneth P. Dietrich School of Arts and Sciences,6400,104257,FISHER SCIENTIFIC COMPANY LLC,PO BOX 406538,NaN,ATLANTA,GA,30384-6538,2344352,6-6400-104257-1,right_only,2344352.0
5861,93,Public Safety and Emergency Management,6400,446194,"LANDMARK EVENT STAFFING SERVICES, INC",4131 HARBOR WALK DRIVE,NaN,FORT COLLINS,CO,80525,1689115,93-6400-446194-1,right_only,1689115.0
1315,30,SVC Health Sciences,6400,114200,QUEST DIAGNOSTICS,P.O. BOX 740709,NaN,ATLANTA,GA,30374-0709,1237840,30-6400-114200-1,right_only,1237840.0
787,10,SVC and Provost,6400,689608,PFL,100 PFL WAY,NaN,LIVINGSTON,MT,59047,790510,10-6400-689608-1,right_only,790510.0
5063,87,SVC and Chief Financial Officer,6400,521322,SUPRA OFFICE SOLUTIONS INC,PO BOX 201,NaN,BALA CYNWYD,PA,19004,600681,87-6400-521322-1,right_only,600681.0
